In [2]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
import scipy.io
import time
from pyDOE import lhs
import tensorflow_probability as tfp
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense



In [ ]:

data = scipy.io.loadmat('C:/Users/Daksh/Desktop/pinn/burgers_shock.mat')
    

In [ ]:

N_u = 100                    # number of data points
N_f = 10000                  # number of points where the differential equation must be satisfied

data = scipy.io.loadmat('burgers_equation.mat')                     # load Burgers equation dataset from matlab file
t = data['t'].flatten()[:, None].astype('float32')                  # time discretization points
x = data['x'].flatten()[:, None].astype('float32')                  # spatial(x) discretization points
exact_sol = np.real(data['usol']).T.astype('float32')               # exact solution

X, T = np.meshgrid(x, t)                                            # create a mesh from data coordinates x,t



In [ ]:
X_domain = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))  # includes spatial in col 1 and then time coordinates col 2
u_domain = exact_sol.flatten()[:, None]

In [ ]:
# Domain bounds
lb = X_domain.min(0)
ub = X_domain.max(0)

In [ ]:
xx1 = np.hstack((X[0:1, :].T, T[0:1, :].T)) # I.C.
uu1 = exact_sol[0:1, :].T                   # u solution for the selected points at diff. conditions as mentioned
xx2 = np.hstack((X[:, 0:1], T[:, 0:1]))  # L.B.C. i.e. at x = -1
uu2 = exact_sol[:, 0:1]
xx3 = np.hstack((X[:, -1:], T[:, -1:])) # R.B.C. i.e. at x = +1
uu3 = exact_sol[:, -1:]

X_u_train = np.vstack([xx1, xx2, xx3])      # Total # of final coordinates (x,t) for the training points
X_f_train = lb + (ub - lb) * lhs(2, N_f)    # coordinates (x,t) of the points to satisfy the differential equation
X_f_train = tf.convert_to_tensor(np.vstack((X_f_train, X_u_train)).astype('float32'))
u_train = np.vstack([uu1, uu2, uu3])        # the solution of the training points

# reducing our dataset by randomly selecting N_u points 
index = np.random.choice(X_u_train.shape[0], N_u, replace=False)
X_u_train = tf.convert_to_tensor(X_u_train[index, :].astype('float32'))
u_train = tf.convert_to_tensor(u_train[index, :].astype('float32'))

# splittling spatial and time coordinates
x_u = X_u_train[:, 0:1]
t_u = X_u_train[:, 1:2]
x_f = X_f_train[:, 0:1]
t_f = X_f_train[:, 1:2]

In [ ]:
# for model u

layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1]

model = Sequential()

h = 2.0 * ( tf.concat([x,t], 1) - lb) / ( ub - lb )  - 1.0

model.add(Dense(layers[0], activation = tf.nn.tanh, input_shape = _))

for l in layers[1:9]:
    model.add(Dense(20, activation = tf.nn.tanh))
    
model.add(Dense( layers[-1]), activation = None)


#def net_u():
    
    
    


In [ ]:
def residual(model_u, x, t):
    with tf.GradientTape() as gtp1:
        gtp1.watch(x)
        with tf.GradientTape(persistent=True) as gtp2:
            gtp2.watch(x)
            gtp2.watch(t)
            u = model_u(x, t)
        du_dt = gtp2.gradient(u, t)
        du_dx = gtp2.gradient(u, x)
    du2_dx2 = tape1.gradient(du_dx, x)
    coefficient = 0.01 / np.pi
    f = du_dt + u * du_dx - coefficient * du2_dx2   # residual
    del gtp2
    return f

In [ ]:
loss = tf.reduce_mean(tf.square(u_tf - u_pred)) + \
        tf.reduce_mean(tf.square(f_pred))



In [ ]:
model.compile(optimizer = 'adam', loss = '', metrics = ['accuracy'])

In [ ]:
model.fit(   ,     , epochs = 600, batch_size = 50, validation split = 0.05)